In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np

# A. 데이터 로드 및 이진 타겟 변수 생성
# 파일 경로를 "test1_all.csv"로 수정합니다.
file_path = "test1_all.csv"
test1_all = pd.read_csv(file_path)


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# 1. 타겟 변수 이진화: PHQ_score를 10점 기준으로 변환
y_binary = (test1_all["PHQ_score"] >= 10).astype(int)

# 2. 독립 변수 X 설정 (PHQ_score는 종속 변수이므로 제거)
X = test1_all.drop("PHQ_score", axis = 1)

# 3. 데이터 분할 및 계층화 (Stratified Split)
# 이진 분류는 클래스 불균형이 중요하므로, 반드시 y_binary를 기준으로 계층화해야 합니다.
X_train, X_valid, y_train_binary, y_valid_binary = train_test_split(
    X,
    y_binary,
    train_size = 0.75,
    random_state=2025,
    stratify = y_binary
)

# 4. 특성 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

print("3번 모델 데이터 준비 완료!")
print(f"훈련 세트 크기: X_train: {X_train.shape}, y_train_binary: {y_train_binary.shape}")
print(f"검증 세트 크기: X_valid: {X_valid.shape}, y_valid_binary: {y_valid_binary.shape}")

3번 모델 데이터 준비 완료!
훈련 세트 크기: X_train: (23003, 88), y_train_binary: (23003,)
검증 세트 크기: X_valid: (7668, 88), y_valid_binary: (7668,)


In [14]:
scaler = StandardScaler()

# 훈련 세트(X_train)에 fit_transform을 적용하여 평균과 표준편차를 학습합니다.
X_train_scaled = scaler.fit_transform(X_train)

# 검증 세트(X_valid)에는 훈련 세트에서 학습한 scaler를 사용하여 transform만 적용합니다.
X_valid_scaled = scaler.transform(X_valid)

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import numpy as np

# ***********************************************
# ⭐ 1. 파일 경로 수정 (실제 파일 경로로 변경) ⭐
file_path = "test1_all.csv"
test1_all = pd.read_csv(file_path)
# ***********************************************

# 2. 타겟 변수 이진화 (PHQ score >= 10)
test1_all['is_depressed'] = (test1_all['PHQ_score'] >= 10).astype(int)
y_binary = test1_all['is_depressed']

# 3. ⭐ 타겟 누수 해결: PHQ 개별 문항과 총점을 모두 제거한 X_clean 정의 ⭐
phq_items_to_drop = [col for col in test1_all.columns if col.startswith('BP_PHQ_')]
columns_to_drop = ['PHQ_score', 'is_depressed'] + phq_items_to_drop

X_clean = test1_all.drop(columns=columns_to_drop, errors='ignore')

# 4. 데이터 분할 (X_clean 사용)
X_train_clean, X_valid_clean, y_train_binary, y_valid_binary = train_test_split(
    X_clean,
    y_binary,
    train_size=0.75,
    random_state=2025,
    stratify=y_binary
)

# 5. 특성 스케일링
scaler = StandardScaler()
X_train_scaled_clean = scaler.fit_transform(X_train_clean)
X_valid_scaled_clean = scaler.transform(X_valid_clean)

# 6. 모델 객체 생성 및 학습
xgb_model_cls = XGBClassifier(
    max_depth=4,
    learning_rate=0.05,
    n_estimators=500,
    objective='binary:logistic', # 이진 분류
    eval_metric='logloss'
)

# ⭐ 학습 ⭐
xgb_model_cls.fit(X_train_scaled_clean, y_train_binary)

# 7. 예측 및 평가
pred_xgb_cls = xgb_model_cls.predict(X_valid_scaled_clean)
pred_proba_xgb_cls = xgb_model_cls.predict_proba(X_valid_scaled_clean)[:, 1]

accuracy = accuracy_score(y_valid_binary, pred_xgb_cls)
roc_auc = roc_auc_score(y_valid_binary, pred_proba_xgb_cls)
f1 = f1_score(y_valid_binary, pred_xgb_cls)

print("\n=== 모델 3번 (이진 분류) - XGBoost Classifier 최종 결과 ===")
print(f"XGBoost Accuracy: {accuracy:.4f}")
print(f"XGBoost ROC AUC: {roc_auc:.4f}")
print(f"XGBoost F1-Score: {f1:.4f}")


=== 모델 3번 (이진 분류) - XGBoost Classifier 최종 결과 ===
XGBoost Accuracy: 0.7838
XGBoost ROC AUC: 0.8333
XGBoost F1-Score: 0.5872


In [28]:
from sklearn.ensemble import RandomForestClassifier

print("=" * 50)
print("### 1. Random Forest Classifier (변수 중요도 해석)")

rf_model = RandomForestClassifier(
    n_estimators=400, max_depth=12, min_samples_split=4,
    min_samples_leaf=2, random_state=42
)
rf_model.fit(X_train_scaled_clean, y_train_binary)

y_pred_rf = rf_model.predict(X_valid_scaled_clean)
y_proba_rf = rf_model.predict_proba(X_valid_scaled_clean)[:, 1]

# 성능 계산
f1_rf = f1_score(y_valid_binary, y_pred_rf)
roc_auc_rf = roc_auc_score(y_valid_binary, y_proba_rf)

print(f"F1-Score: {f1_rf:.4f}, ROC AUC: {roc_auc_rf:.4f}")

# ⭐ 변수 중요도 해석 ⭐
importances_rf = pd.Series(rf_model.feature_importances_, index=X_train_clean.columns)
top_importances_rf = importances_rf.sort_values(ascending=False).head(10)
print("\n[변수 중요도 해석 (Top 10):]")
print(top_importances_rf)

### 1. Random Forest Classifier (변수 중요도 해석)
F1-Score: 0.5491, ROC AUC: 0.8311

[변수 중요도 해석 (Top 10):]
BP1          0.137890
LQ_5EQL      0.105301
D_1_1        0.071996
age          0.035757
LQ_4EQL      0.027107
D_2_wk       0.024533
EC_lgw_2     0.024439
EC_wht_23    0.023781
BE3_walk     0.022785
BE8_sit      0.021153
dtype: float64


In [29]:
from sklearn.linear_model import LogisticRegression

print("=" * 50)
print("### 2. Logistic Regression (가중치 해석)")

log_model = LogisticRegression(solver='liblinear', random_state=2025)
log_model.fit(X_train_scaled_clean, y_train_binary)

y_pred_log = log_model.predict(X_valid_scaled_clean)
y_proba_log = log_model.predict_proba(X_valid_scaled_clean)[:, 1]

# 성능 계산
f1_log = f1_score(y_valid_binary, y_pred_log)
roc_auc_log = roc_auc_score(y_valid_binary, y_proba_log)

print(f"F1-Score: {f1_log:.4f}, ROC AUC: {roc_auc_log:.4f}")

# ⭐ 가중치(계수) 해석 ⭐
coef_log = pd.Series(log_model.coef_[0], index=X_train_clean.columns)
top_coefs_log = coef_log.sort_values(ascending=False).head(10)
print("\n[가중치 해석 (Top 10):]")
print(top_coefs_log)

### 2. Logistic Regression (가중치 해석)
F1-Score: 0.5795, ROC AUC: 0.8296

[가중치 해석 (Top 10):]
EC1_1      2.294520
BP6_31     1.610372
marri_2    1.217672
LQ4_15     0.887048
LQ_5EQL    0.426256
BD1        0.408117
D_1_1      0.369989
sex        0.323520
BD7_4      0.267351
occp       0.160553
dtype: float64


In [35]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
# FrozenEstimator는 코드를 실행하는 환경에 따라 Import 위치가 달라질 수 있습니다.
# from sklearn.utils._testing import FrozenEstimator

print("=" * 50)
print("### 3. Support Vector Machine (선형 SVM) - 최종 코드")

# LinearSVC 모델 학습
svm_model = LinearSVC(C=1.0, random_state=2025, dual=False)
svm_model.fit(X_train_scaled_clean, y_train_binary)

# F1-Score는 직접 예측으로 계산
y_pred_svm = svm_model.predict(X_valid_scaled_clean)

# ROC AUC 계산을 위해 확률 값 보정 (FrozenEstimator를 사용하여 경고를 무시하고 진행)
calibrated_svm = CalibratedClassifierCV(svm_model, method='sigmoid', cv=3)
calibrated_svm.fit(X_valid_scaled_clean, y_valid_binary)
y_proba_svm = calibrated_svm.predict_proba(X_valid_scaled_clean)[:, 1]

# 성능 계산
f1_svm = f1_score(y_valid_binary, y_pred_svm)
roc_auc_svm = roc_auc_score(y_valid_binary, y_proba_svm)

print(f"F1-Score: {f1_svm:.4f}, ROC AUC: {roc_auc_svm:.4f}")

# 가중치(계수) 해석
coef_svm = pd.Series(svm_model.coef_[0], index=X_train_clean.columns)
top_coefs_svm = coef_svm.sort_values(ascending=False).head(10)
print("\n[가중치 해석 (Top 10):]")
print(top_coefs_svm)
print("=" * 50)

### 3. Support Vector Machine (선형 SVM) - 최종 코드
F1-Score: 0.5626, ROC AUC: 0.8360

[가중치 해석 (Top 10):]
marri_2    0.862897
BP6_31     0.776865
EC1_1      0.686932
LQ4_15     0.365848
LQ_5EQL    0.169465
BD1        0.151566
D_1_1      0.130871
sex        0.114646
BD7_4      0.096335
LQ2_ab     0.046176
dtype: float64
